# YORUMLARIN ÖZETLENMESI
 Word2Vect ve CountVectorizer yardımıyla Otele ait en çok bahsedilen 10 özelliğin öne çıkan yorumlarını bulmak
Projemiz sayıları binlerle ifade edilen yorumları özetlemeyi amaçlamaktadır. Proje kapsamında bir isletmeye ait kullanıcı yorumlarının kısa ve öz halini isletme sahibine sunmayı, bu yolla işletme sahibinin işletmesi hakkındaki kullanıcı yorumlarının özet halini görmesini ve işletmesinideki olumlu ve olumsuz yanları kısa sürede öğrenmesini gerçekleştirmektedir.

   Bu proje kapsamında kullanıcı veritabanından kullanıcı oteline ait yorumların bulunduğu döküman alındı. Daha sonra bu dökümandan yorumlar çıkartıldı. Yorumlar üzerinde lovercase, stopwords'ün ayıklanması ve lemmazination gibi bazı ön işlemler yapıldı. Ön işleme tabi tutulan yorum bilgileriyle word2vec yordamıyla bir sözlük oluşturduk daha sonra bu sözlük üzerinde daha önce belirlediğimiz 10 özelliği aratıp bu özellik yerine geçebilecek benzer kelimeleri tespit ettik. Her bir özellik için benzer ilk 5 kelimeyi bulup kaydettik.
   
# Word2Vec Tekniği
Word2Vec kelimeler arasındaki uzaklık durumlarının vektörel olarak bulunmasına yarayan algoritmalar bütünüdür [3]. Vektörel yapı üzerinde yazılmış araçlar ile bir kelimeye yakın veya uzak kelimeleri listeleme işi yapılabilir. Belli kelimelerin aynı cümle içerisinde sık kullanımları bu vektörel yapının üzerinde bu iki kelimenin birbirine yakınlığını arttırır. Bu vektörel yapının üzerine yazılmış araçlar ile bir kelimeye en yakın kelimeler listenebilir, bu yöntemle projenin amacı olan özetleme kısmı gerçekleştirilebilir.

[3] W. Ling, C. Dyer, A. W. Black, and I. Trancoso, “Two/too simple adaptations of word2vec for syntax problems,” in Proceedings of the 2015 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, 2015, pp. 1299–1304.

# CountVectorizer Tekniği
CountVectorizer, hem metin belgeleri koleksiyonunu belirtmek hem de bilinen kelimelerin bir sözlüğünü oluşturmak için basit bir yol sağlar. Sözlüğün uzunluğunda bir vektör döndürülür ve her kelimenin belgede kaç kez göründüğüne ilişkin bir tam sayı vektör içinde yer alır. Oluşturulan sözlükten belirlenen kelime ikili/üçlülerinin beraber sık kullanma oranlarına göre sıralar.


# Otel Özelliklerinin Belirlenmesi
Ilk hedef otel ile ilgili bakılacak/dikkat edilecek özellikleri belirlemekti. Bunu belirlerken a¸sa˘gıdaki maddeler göz önünde bulunduruldu.
        * En fazla bahsedilen kelimeler nelerdir?
        * Belirlenen özellikler, otel ile alakalı nesneler veya otele ait bir hizmet midir? 

Bu iki madde göz önünde bulundurularak aşağıdaki kelimeler otel özellikleri olarak belirlendi.
1.  staff 
1.  location 
1.  room 
1.  breakfast 
1.  bed 
1.  service 
1.  bathroom 
1.  restaurant
1.  view 
1.  food 

In [ ]:
import re, nltk, os, gensim
import pandas as pd  # data processing, CSV file I/O // pd.read_csv
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [ ]:

stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
# Yandakiler bir kereye mahsus yüklenmeli : nltk.download('stopwords')  /  nltk.download('punkt')  /  nltk.download('wordnet')


1. Üstte yazılan kütüphanelerin bazıları tek tek el ile manuel install edilmeli.
>     python -m pip install library_name
2. Aşağıdakiler bir kereye mahsus yüklenmeli:
>    nltk.download('stopwords')   
>    nltk.download('punkt')  
>    nltk.download('wordnet')
3. Dataseti indirilip iki dosyadada konumu belirtilmeli. (/ işaretinin yönüne dikkat edilmeli.)
Link: [ https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe ]

In [ ]:
df = pd.read_csv('../input/Hotel_Reviews.csv') #Buraya dosya konumunu ve ismini yazacaksın: 
                                               #'C:/Users/PycharmProjects/Hotel_Reviews_n40.csv') gibi burada "/" yönüne dikkat et!
print(df.shape)  # Veritabanının satır/sütun bilgisi

In [ ]:

df['State'] = df.Hotel_Address.apply(lambda x: x.split(' ')[-1])  # Adress uzun olduğu için sadece ülke bilgisini aldık.
print(df.Hotel_Name.nunique(), 'hotels belonging to', df.State.unique().size,'different Country in the dataset')
print('Country : ', df.State.unique())
print('\t\t\t\t\t--------Hotels--------\n', df['Hotel_Name'].unique())

* Aşağıdaki yoruma alınan iki satırda otel ismi sorulur ve verilen otele ait bilgilerle işlem yapılır. Şimdilik bunu kaldırıp 3. satırla birlikte tüm otelleri DtaFrame'me alıyorum. Mevcut veri setimizde bir sürü sütun vardı biz lüzumsuz olan sütunları çıkartıp sadece lazım olabilecekleri alıp yeni bir DataFrame'e koyduk.

In [ ]:
#isim = input("Please enter hotel name?")
#df_com = df[(df.Hotel_Name == isim)][['Hotel_Name','Reviewer_Score','Negative_Review','Review_Total_Negative_Word_Counts','Positive_Review','Review_Total_Positive_Word_Counts','Total_Number_of_Reviews','Total_Number_of_Reviews_Reviewer_Has_Given']]
df_com = df[['Hotel_Name','Reviewer_Score','Negative_Review','Review_Total_Negative_Word_Counts','Positive_Review','Review_Total_Positive_Word_Counts','Total_Number_of_Reviews','Total_Number_of_Reviews_Reviewer_Has_Given']]


#                               ÖN İŞLEMLER
**Aşağıdaki kodda Pozitif ve negatif yorumlarımızı bazı ön işlemlerden geçiriyoruz. Bu işlemler:**
1.  Noktalama işaretlerini kaldır. Sadece harfler kalsın
1.  Tokenize : Metindeki herbir kelimeyi ayrı birer string yapar kelimeler listesi yapar. ['The', 'hotel', 'was', 'perfect'] gibi
1.  Lowercase : Tüm karakterleri küçülttük
1.  Out of Stopwords : Stopwords’lerin(Mevcudiyetinin çok bir anlamı olmayan kelimelerin) kaldırılması. The, am, are, we, she, at, in, or, and gibi
1.  Lemmazination : Kelimeleri özüne döndürme işlemini yapar. Textte bulunan çoğul kelimelerin tekil hale getirilmesi gibi. Aslında beklentimiz çok daha fazlası ama yapamıyor.(örn. -ing eklerini kaldırması, fillerin 2. ve 3. hallerinden aslına döndürmesi vs.)

In [ ]:
# NLP prepross NEGATIVE reviews
neg  = []
negc = []
for i in df_com['Negative_Review']:    #i:   The hotel was perfect
    letters = re.sub('[^a-zA-Z]', ' ', i)  # sadece harflerden oluşsun noktalamaları/sayıları kaldır. #letters:   The hotel was perfect
    tokens = nltk.word_tokenize(letters)  # herbir kelimeyi ayrı birer string yapar kelimeler listesi yapar #tokens:  ['The', 'hotel', 'was', 'perfect']
    lowercase = [l.lower() for l in tokens] #lowercase:  ['the', 'hotel', 'was', 'perfect']
    filtered_result = list(filter(lambda l: l not in stop_words, lowercase))    #filtered_result:  ['hotel', 'perfect']
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]  # sondaki çoğul eki kaldırır  #lemmas:  ['hotel', 'perfect']
    neg.append(' '.join(lemmas))  # dizinin sonuna ekle
    negc.append(lemmas)

# NLP prepross positive reviews
pos  = []
posc = []
for i in df_com['Positive_Review']:
    pletters = re.sub('[^a-zA-Z]', ' ', i)
    ptokens = nltk.word_tokenize(pletters)
    plowercase = [l.lower() for l in ptokens]
    pfiltered_presult = list(filter(lambda l: l not in stop_words, plowercase))
    plemmas = [wordnet_lemmatizer.lemmatize(t) for t in pfiltered_presult]
    pos.append(' '.join(plemmas))
    posc.append(plemmas)

top = []
top = negc + posc; # Bu diziyi Word2Vec'i oluştururken kullanacağız.

 #   Word2Vect
 
Öncelikle Word2Vect modeli ile bir vocabulary oluşturulur. Daha önce belirlenen 10 kelime Word2Vect fonksiyonuna gönderilir ve bu kelimelere yakın anlamlı 5’er kelime bulunur.
 
Mesela staff, oteli tanımak için kullanıcıların en çok yorum yaptığı bir otel özelliğidir. Kullanıcılar staff yerine stafı kasteden farklı ifadeler de kullanmışlardır. Bu farklı kullanımları Word2Vect yardımıyla keşfedip en çok kullanılan 5 tanesini dstaff isimli bir diziye atıyoruz. Bu işlem diğer oteli tanımlayan tüm özellikler için de yapılır. Sonuç olarak elimizde oteli tanımlayan 10 adet özellik ve bu özellikler yerine kullanıcıların kullandığı ifadeler dizisi olmuş olur.

In [ ]:
model = gensim.models.Word2Vec(top, size=150, window=10, min_count=2, workers=10)
model.train(top, total_examples=len(top), epochs=10)

def word2vectfonc(kdizi, k):
    print( str(k) +". Most similar to {0}".format(kdizi), model.wv.most_similar(positive=kdizi, topn=5))
    dizi = model.wv.most_similar(positive=kdizi, topn=5)
    for i in dizi:
        kdizi.append(i[0])

dstaff = ['staff']
dloc = ["location"]
droom = ['room']
dbreakfast = ["breakfast"]
dbed = ["bed"]
dservice = ["service"]
dbath = ["bathroom"]
dview = ["view"]
dfood = ["food"]
drest = ["restaurant"]

word2vectfonc(dstaff, 1)
word2vectfonc(dloc, 2)
word2vectfonc(droom, 3)
word2vectfonc(dbreakfast, 4)
word2vectfonc(dbed, 5)
word2vectfonc(dservice, 6)
word2vectfonc(dbath, 7)
word2vectfonc(dview, 8)
word2vectfonc(dfood, 9)
word2vectfonc(drest, 10)


# Count Vectorizer
Daha sonra CountVectorizer yardımıyla olumlular için en çok geçen kelime ikililerine, olumsuzlar için ise kelime 3’lülerine bakılır ve arka planda ilk 100 ikili/üçlü listelenir. Listelenen kelimelere, daha önce w2v yardımıyla belirlenen 10 kelimeye ait kendilerininde içinde bulunduğu dizi gönderilir ve eşleşen yorumlar bulunarak ilgili kelime ve ona yakın anlamlı kelimelere ait başlık altında, en sık bahsedilenler olarak yer alır. 

In [ ]:

# Finding most important words in Negative Reviews
print(' ')
cv = CountVectorizer(analyzer='word', stop_words='english', max_features=100, ngram_range=(3, 3))
# Kelime kelime n gram bunu harf harf te yapabiliriz. Bunu analyses keyi ile belirliyoruz. Ve n-gram buna göre yapılacak
# The lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted.
# All values of n such such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams,
# (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams. Only applies if analyzer is not callable.
print(cv)  # hangi özelliklerde bir vectorizer olacak
most_negative_words = cv.fit_transform(neg)
# print(cv.get_feature_names()) #['bed bit hard', 'breakfast included price', 'breakfast room small', 'building work going', 'coffee making facility', ...
# print(most_negative_words) #(35, 15)	1 /(80, 9)	1  / (35, 15)	1
# print(most_negative_words.size)  # 605 tane var
temp_counts = most_negative_words.sum(axis=0)  # axis=0 demek 0 olan yerleri none yapma 0 olarak hesaba kat demek.
# print(temp_counts) #[[15 33 16 14 22 16 14 17 14 86 13 24 17 70 58 23 16 19 24 15 13 13 16 20  17]]
temp_words = cv.vocabulary_  # {'room really small': 15, 'room bit small': 9, 'bed bit hard': 0, 'room size small': 16, 'tea coffee room': 24,
print('the most important words in Negative Reviews: \n--------------------------------------------')
dizii2 = []


In [ ]:
def negfonks(dizii):
    x = 0
    for key, value in dict.items(temp_words):
        for i in dizii:
            if i in key and x < 8:
                print(dizii[0], " :", key, "   -", i)
                x = x + 1
                dizii2.append(key)

negfonks(dstaff)
negfonks(dstaff)
negfonks(dloc)
negfonks(droom)
negfonks(dbreakfast)
negfonks(dbed)
negfonks(dservice)
negfonks(dbath)
negfonks(dview)
negfonks(dfood)
negfonks(drest)
#display(temp_words)


Mevcut çıktılarda CountVectorizer’ın çalışması sonucunun ekran çıktısı verilmiştir. Her bir satır 3 nesneden oluşmaktadır. İki noktaya kadar olan kelime daha önce otel için belirlenen özelliği temsil etmektedir. İki noktadan sonraki ikili/üçlü kelimeler ilgili özelliğin en çok bahsedilen niteliğini belirtmektedir. Köşeli paratezler arasındaki kısım ilgili niteliğin hangi kelime ile eşleşmesi sonucunda geldiğini göstermektedir. 
Bazı örneklerde parantez içinle ilk kısımdaki kelimeler farklıdır. Bu farklı olanlar ilgili özetin ana özellikten değil, ana özellikle yakın anlamlın olan kelimelerden geldiğini belirtmektedir. Bir olayı incelemek gerekirse bathroom kelimesi ve bu kelimeye yakın beş anlamlı kelime gönderildiğinde, en çok tekrar eden 8 özellik listelenmektedir. [bath] ve[shower] içeren satırlar ilgili niteliğin bathroomdan değil, bathroom kelimesiyle yakın anlamlı olan shower ve bath kelimesinden geldiğini göstermektedir.

In [ ]:
print('                                          ')
cv = CountVectorizer(analyzer='word', stop_words='english', max_features=100, ngram_range=(2, 2))
most_positive_words = cv.fit_transform(pos)
temp1_counts = most_positive_words.sum(axis=0)
temp1_words = cv.vocabulary_

print('The most important words in Positive Reviews: \n--------------------------------------------')
dizii1 = []


In [ ]:

def posfonks(dizii):
    x = 0
    y = 0
    for key, value in dict.items(temp1_words):
        for i in dizii:
            if i in key and x < 8:
                print(dizii[0], " :", key, "   -", i)
                dizii1.append(key)
                x = x + 1

posfonks(dstaff)
posfonks(dloc)
posfonks(droom)
posfonks(dbreakfast)
posfonks(dbed)
posfonks(dservice)
posfonks(dbath)
posfonks(dview)
posfonks(dfood)
posfonks(drest)
#display(temp1_words)

Sonuç olarak daha önce belirlenen otele ait 10 özellik için en çok bahsedilen olumlu ve olumsuz yorumlar listelenmiş olur. Böylelikle işletmeci oteli hakkında kullanıcıların hangi özellikleri sevip hangi özelliklerini sevmediğini kısa sürede özet olarak öğrenmiş olur.
